#**Space-based Tokenization**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/clean_shakespeare.txt'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import nltk
from collections import Counter

with open (file_path, 'r') as f:
  text = f.read()

def get_words(text):

  pattern = r'''(?x)          # set flag to allow verbose regexps
          (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
          | \w+'\w+           # contractions
        | \w+(?:-\w+)*        # words with optional internal hyphens
        | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
        | \.\.\.              # ellipsis
        | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
      '''

  vocab = nltk.regexp_tokenize(text, pattern)
  vocab = [word.lower() for word in vocab]
  unique_words = set(vocab)
  c = Counter(vocab)

  sorted_dict = {key: value for key, value in sorted(
      c.items(), key=lambda item: item[1], reverse=True)}

  return sorted_dict

# Byte Pair Encoding

- make Shakespeare into test 1% and train
- train the segmenter with varying k (try normalization strategies)
- compare the performance against a different set
- figure out a measure for accuracy

In [8]:
from itertools import islice

def performance(sorted_dict, vocab, k=10000):

    def take(n, iterable):
        """Return the first n items of the iterable as a list."""
        return list(islice(iterable, n))


    # extract top k keys
    qualifier = [key for key, _ in take(k, sorted_dict.items())]

    # clean vocab by stripping the _
    cleaned_vocab = {word.rstrip('_') for word in vocab}

    # calculate overlap
    overlap = set(qualifier) & cleaned_vocab
    num_overlap = len(overlap)

    # percentage
    percentage = (num_overlap / len(qualifier)) * 100
    return percentage


In [28]:
import numpy as np
from collections import defaultdict

from collections import OrderedDict
from collections import Counter


def bpe(dictionary, k=None):
  """
  Input:
    dictionary (dict): a dictionary that contains the tokens and their respective counts
  return:
    vocab_bpe (list): vocabulary of the corpus
    sorted_token_freq:
    dict_matrix:
  """

  # get all unique characters in original corpus
  all_keys = "_ ".join(dictionary.keys())
  vocab_bpe = list(set(all_keys))

  # Corpus/dictionary in einzelne tokens splitten, nach jedem Wort (VOR space!) "_" einfügen
    # worte als list of characters
  dict_matrix = []
  for key, value in dictionary.items():
    new_key = list(f"{str(key)}_ ")
    dict_matrix.append([new_key, value])


  # NICHT corpus, sondern liste an Wörtern in einzelne tokens splitten,
  # >> jede occurence mit counts der Worte multiplizieren


  token_freq = defaultdict(int) # TODO: does that need to be moved outside of the loop?
  iteration = True
  num_rounds = 0
  while iteration:

    for token_list, value in dict_matrix:
      for i in range(len(token_list)-2):
        # wollen den und den nächsten token als key
        search_key = token_list[i] + token_list[i+1]
        # zu dictionary hinzufügen falls key noch nicht existiert
        token_freq[search_key] += value
    # word_freqs: gehen jede existierende Folge aus zwei tokens in list of words von vorne bis hinten durch

    c = Counter(token_freq)

    sorted_token_freq = {key: value for key, value in sorted(
        c.items(), key=lambda item: item[1], reverse=True)}

    # Find the most frequent token not already in vocab_bpe
    for token in sorted_token_freq.keys():
      if token not in vocab_bpe:
        first_token = token
        break
      else:
        first_token = None  # Optional: fallback in case all tokens are already in vocab

    if first_token:
      vocab_bpe.append(first_token)
    else:
      print("No new token to add.")

    # höchster count wird gemerged:
    # add to vocab
    #first_token = list(sorted_token_freq.keys())[0]
    #vocab_bpe.append(first_token)
    # replace in list of words
    # start again?
    for i in range(len(dict_matrix)):
        token_list, value = dict_matrix[i]
        j = 0
        while j < len(token_list) - 1:
            search_key = token_list[j] + token_list[j + 1]
            if search_key == first_token:
                merged_token = search_key
                # Merge the tokens
                token_list = token_list[:j] + [merged_token] + token_list[j + 2:]
                # Don't increment j — might be able to merge again
            else:
                j += 1
        dict_matrix[i][0] = token_list
    if k:
       k -= 1
       iteration = (k > 0)

    else:
       num_rounds += 1
       accuracy = performance(dictionary, vocab_bpe, 500)
       iteration != (accuracy > 70)

       if num_rounds > 1500:
          print("exceeded, accuracy: ", accuracy)


          iteration = False

  return vocab_bpe, sorted_token_freq, dict_matrix

In [29]:
with open (file_path, 'r') as f:
  text = f.read()

#split into test and train 10% to 90%
text_train = text[:int(len(text)*0.99)]
text_test = text[int(len(text)*0.99):]

In [30]:
dict_train = get_words(text_train)
dict_test = get_words(text_test)

vocab_train, sorted_token_freq_train, dict_matrix_train = bpe(dict_train)

exceeded, accuracy:  75.6


In [31]:
print(f"Shakespeare vocabulary: {vocab_train}")
print(f"Shakespeare token_freq: {sorted_token_freq_train}")
print(f"Shakespeare dict_matrix: {dict_matrix_train}")

Shakespeare vocabulary: [':', '3', ']', '7', 'g', 'a', '0', '`', ',', 'k', '.', '1', ' ', 'y', 'm', 'c', '8', 't', 'f', 'b', '4', '6', '(', '5', 'q', '?', 'u', "'", 'r', ';', 'z', '_', 'n', 'l', 'v', '"', '9', 'j', ')', 'x', '-', 'e', 'i', 'd', 's', 'w', 'h', '[', 'p', '2', 'o', 'e_', 'th', 's_', 't_', 'd_', ',_', '._', 'r_', 'an', 'y_', 'n_', 'ou', 'in', 'o_', 'er', 'en', 'l_', 'or', 'ar', 'ha', 'ea', 'f_', 'the_', 'on', 'll', 'he', 'no', 'is_', 'i_', 're', 'and_', 'hi', 'a_', 'er_', 'the', 'st', 'wi', 'es', 'm_', 'th_', ';_', 'al', 'g_', 'me_', 'you', 'ro', 'ow', 'at_', 'to_', 'el', 'll_', 've_', 'ch', 'om', 'te', 'es_', 'in_', 'le', 'ti', 'st_', 'fo', 'li', 'lo', 'ma', 'of_', 'ou_', 'ri', 'h_', 'my_', 'k_', 'on_', 'sh', '?_', 'si', 'ce_', 'se', 'it_', 'or_', 'w_', 'ra', 'ee', 'se_', 'at', 'is', 'wh', 'be', 'ur', 'la', 'an_', 'to', 'bu', 'oo', 'us_', 'as_', 'co', 'gh', "'d_", "'_", 'he_', 'ing_', 'ol', 'ld_', 'di', 'you_', 'ay_', 'ho', 'un', "'s_", 'ke_', 'de', 'en_', 'not_', 'be_', 

In [27]:
train_accuracy = performance(dict_train, vocab_train, 500)
test_accuracy = performance(dict_test, vocab_train, 500)

print("train accuracy: ", train_accuracy)
print("test accuracy: ", test_accuracy)

train accuracy:  100.0
test accuracy:  69.6


In [18]:
import tiktoken

def compare_to_gpt_encoding(text, alphabet, model_name="gpt-3.5-turbo"):
    """
    Encodes a text string using tiktoken and prints the tokens along with their IDs.

    Args:
        text: The text string to encode.

    """
    alphabet = {word.rstrip('_') for word in alphabet}
    try:
        encoding = tiktoken.encoding_for_model(model_name)
    except KeyError:
        print(f"Warning: Model '{model_name}' not found.  Using 'cl100k_base' instead.")
        encoding = tiktoken.get_encoding("cl100k_base")

    tokens = encoding.encode(text)
    tp = 0

    unique_tokens = set(tokens)
    for token in unique_tokens:
      decoded_token = encoding.decode([token])
      #check if the decoded token is in the dict alphabet
      if(decoded_token in alphabet):
        tp = tp+1
    total_len = len(alphabet)
    print(tp/total_len)


with open (file_path, 'r') as f:
  text = f.read()
model_name="gpt-3.5-turbo"
tokens_train = compare_to_gpt_encoding(text_train, vocab_train, model_name)
tokens_test = compare_to_gpt_encoding(text_test, vocab_train, model_name)

0.6382818387339865
0.15900527505651846


# Different Corpus: Friends TV Show

In [19]:
with open ('friends.txt', 'r') as f:
  friends = f.read()

friends_dict = get_words(friends)
friends_accuracy = performance(friends_dict, vocab_train, 500)
print("Accuracy on Friends: ", friends_accuracy)

tokens_friends = compare_to_gpt_encoding(friends, vocab_train, model_name)

FileNotFoundError: [Errno 2] No such file or directory: 'friends.txt'

# Task 2

## N-Grams (first uni-gram, gradual increasing to 4, have n as input, Laplace-Smoothing)
## Perplexity

## Interpolation vs Backoff

## Generator

In [78]:
import numpy as np

def to_byte_pair(context, vocab):
    context = context.replace(' ', '_')
    context = context.lower()
    # sort vocab
    vocab.sort(key=lambda x: len(x), reverse=True)
    final_list = []
    mismatch = ""
    while context != "":
        old_context = context
        match = False
        for token in vocab:
            if token == context[:len(token)]:
                final_list.append(mismatch)
                final_list.append(token)
                context = context[len(token):]
                mismatch = ""
                match = True
                break
        if not match:
            mismatch += context[0]
            context = context[1:]

        if old_context == context:
            break

    final_list.append(mismatch)
    return final_list

In [79]:
from collections import Counter, defaultdict

class N_gram:

  def __init__(self, corpus, n):
    self.unigram_probs = self.get_unigram_probs(corpus)
    self.split_text = self.split_ngrams(corpus, n)
    #self.n_gram_probs = self.n_gram_probs(self.split_text, n)


  def get_unigram_probs(self, corpus):
    """
    Pass corpus (already to byte-pair) and get unigram probs.
    """
    c = Counter(corpus)
    total = sum(c.values())
    unigram_probs = {token: count / total for token, count in c.items()} # is that normalised the way we want it
    return unigram_probs


  def split_ngrams(self, corpus, n):
    """
    Pass corpus (already to byte-pair)
    Funtion to split that corpus into n-grams (n tokens are chunked together)
    """
    split_text = []
    for i in range(len(corpus) - n + 1):
      split_text.append(corpus[i : i+n])

    return split_text


  def calculate_n_gram_probs(self, split_text, n):
    """
    Builds conditional probabilities: P(w_n | w_1, ..., w_{n-1})
    Currently uses nested dictionaries, need to make that more useful for you
    """
    # nested defaultdicts for automatic initialization
    n_gram_counts = defaultdict(lambda: defaultdict(int))

    for n_gram in split_text:
      prefix = tuple(n_gram[:-1])  # context: first n-1 tokens
      target = n_gram[-1]          # prediction target: nth token
      n_gram_counts[prefix][target] += 1

    # convert counts to probabilities
    n_gram_probs = {}

    for prefix, target_counts in n_gram_counts.items():
      total = float(sum(target_counts.values()))
      n_gram_probs[prefix] = {token: count / total for token, count in target_counts.items()}


    return n_gram_probs


In [80]:
# make corpus to byte_pairs
n_gram_corps = to_byte_pair(text_train[:100000], vocab_train)

print(n_gram_corps)

# idea would be to pass n_gram_corps to the unigram function

['\n', '1', '', '6', '', '0', '', '9', '\n\n', 'the_', '', 'son', '', 'ne', '', 't', '', 's', '\n\n', 'by_', '', 'will', '', 'ia', '', 'm_', '', 'sha', '', 'ke', '', 'spea', '', 're', '\n\n\n\n', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '_', '', '1', '\n', '_', '', '_', '', 'from_', '', 'fai', '', 'rest_', '', 'cre', '', 'at', '', 'ur', '', 'es_', '', 'we_', '', 'des', '', 'ire_', '', 'inc', '', 'rea', '', 'se', '', ',', '\n', '_', '', '_', '', 'that_', '', 'there', '', 'by_', '', 'beau', '', 'ty', '', "'s_", '', 'ros', '', 'e_', '', 'might_', '', 'never_', '', 'di', '', 'e', '', ',', '\n', '_', '', '_', '', 'but_', '', 'as_', '', 'the_', '', 'ri', '', 'per_', '', 'should_', '', 'by_', '', 'time_', '', 'de', '', 'ce', '', 'as', '', 'e', '', ',', '\n', '_', '', '_', '', 'his_', '', 'ten', '', 'der_', '', 'he', '', 'ir_', '', 'might_', '', 'bear_', ''

In [81]:
my_unigram = N_gram(n_gram_corps, 1)
my_bigram = N_gram(n_gram_corps, 2)

print(f"Unigram probabilities: {my_unigram.unigram_probs}")
print(f"Unigram split: {my_unigram.split_text}")
print(f"Bigram split: {my_bigram.split_text}")
print(f"Bigram probabilities: {my_bigram.calculate_n_gram_probs(my_bigram.split_text, 2)}")


my_bigram_probs = my_bigram.calculate_n_gram_probs(my_bigram.split_text, 2)

#print(max(my_bigram_probs, key=my_bigram_probs.get))


Unigram probabilities: {'\n': 0.027360014711097683, '1': 0.0011427370522638016, '': 0.47034794373005134, '6': 0.00034150762481446943, '0': 0.00034150762481446943, '9': 0.00034150762481446943, '\n\n': 0.0001050792691736829, 'the_': 0.005608605992145324, 'son': 0.0002495632642874969, 'ne': 0.0010639276003835394, 't': 0.002718926089869045, 's': 0.0025350373688151, 'by_': 0.0011427370522638016, 'will': 0.000288967990227628, 'ia': 6.56745432335518e-05, 'm_': 0.0004203170766947316, 'sha': 0.0007092850669223595, 'ke': 0.0005648010718085455, 'spea': 0.00011821417782039326, 're': 0.0007880945188026218, '\n\n\n\n': 1.3134908646710362e-05, '_': 0.10846807560453417, 'from_': 0.0010507926917368289, 'fai': 0.0003152378075210487, 'rest_': 0.0001313490864671036, 'cre': 0.00023642835564078652, 'at': 0.00019702362970065544, 'ur': 0.0005648010718085455, 'es_': 0.001274086138730905, 'we_': 0.00019702362970065544, 'des': 0.0005516661631618352, 'ire_': 7.880945188026217e-05, 'inc': 0.000144483995113814, 're

In [83]:
def get_top_bigrams(n_gram_probs, k=5):
    bigram_list = []

    for context, next_words in n_gram_probs.items():
        for word, prob in next_words.items():
            bigram = context + (word,)
            bigram_list.append((bigram, prob))

    # Sort by probability (descending)
    bigram_list.sort(key=lambda x: x[1], reverse=True)

    # Print top k
    print(f"Top {k} most probable bigrams:")
    for i in range(min(k, len(bigram_list))):
        bigram, prob = bigram_list[i]
        print(f"{bigram} → {prob:.4f}")

get_top_bigrams(my_bigram_probs, 20)

Top 20 most probable bigrams:
('the_', '') → 1.0000
('son', '') → 1.0000
('ne', '') → 1.0000
('by_', '') → 1.0000
('ia', '') → 1.0000
('m_', '') → 1.0000
('sha', '') → 1.0000
('ke', '') → 1.0000
('spea', '') → 1.0000
('\n\n\n\n', '_') → 1.0000
('from_', '') → 1.0000
('fai', '') → 1.0000
('rest_', '') → 1.0000
('cre', '') → 1.0000
('at', '') → 1.0000
('ur', '') → 1.0000
('es_', '') → 1.0000
('we_', '') → 1.0000
('des', '') → 1.0000
('ire_', '') → 1.0000
